<a href="https://colab.research.google.com/github/jiieunshin/lecture_deeplearning/blob/master/bert_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 이는 구글 코랩으로 돌린 버전입니다. 그리고 기본 코드는 SKT Brain의 KoBERT를 그대로 가져왔고, 학습 및 테스트 데이터셋만 따로 준비한 것입니다.
# SKT Brain github 주소는 다음과 같습니다. https://github.com/SKTBrain/KoBERT
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3 # 최신 버전으로 설치하면 "Input: must be Tensor, not str" 라는 에러 발생
!pip install torch
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

     |████████████████████████████████| 46.9MB 65kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 7.0MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595699 sha256=6e7f87fa3f3325295ad738937278cec49b490162e583b893cd94e38f95a52230
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.2MB 8.3MB/s 
     |████████████████████████████████| 757kB 8.1MB/s 
     |████████████████████████████████| 901kB 50.3MB/s 
     |████████████████████████████████| 3.0MB 44.6MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-i1jt6h12
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-i1jt6h12
  Created wheel for kobert:

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [3]:
##GPU 사용 시
device = torch.device("cuda:0")
bertmodel, vocab = get_pytorch_kobert_model()

# 학습용 데이터셋 불러오기
import pandas as pd
train_data = pd.read_csv("/content/drive/MyDrive/bigdata_deeplearning/data/train_data.csv")
test_data = pd.read_csv("/content/drive/MyDrive/bigdata_deeplearning/data/test_data.csv")

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [4]:
train_data['카테고리'].value_counts()

4    1963
3    1897
2    1457
1    1362
0    1168
Name: 카테고리, dtype: int64

In [5]:
train_data

,청원진행,참여인원,카테고리,청원시작,청원마감,제목,내용
0,청원종료,299,1,2018-05-01,2018-05-31,인천 초등생 살해범 징역 13년이라니요..,네이번 뉴스를 보다 너무 어이가 없어서 청원을 올립니다. 하루가 멀다하고 올라오는 ...
1,청원종료,1676,2,2018-12-13,2019-01-12,우리 거북이 아이들의 미래를 부디 열어주세요,안녕하세요 4세 발달지연 아이를 키우고있는 대한민국의 평범한 어머니입니다 \n\r\...
2,청원종료,384,3,2018-03-19,2018-04-18,단역배우자매를 성폭행한 가해자 12명을 다시 수사하고 담당경찰관을 파면해주세요,10년전의 일이고 고소를 취하한 내용이지만\n이자매가 그일로인해 자살하고 아버지마저...
3,청원종료,188,3,2018-08-26,2018-09-25,은혜로 목사와 그 주변의 관계자들을 인터폴에 문의 하여 체포해 주세요,은혜로 목사인 신목사에게 꼬임을 당해 끌려간 사람들을 구하고 신목사와 그 주변에 ...
4,청원종료,177,2,2017-11-17,2017-12-17,"대구시 우동기 교육감을 중도에 교체할 수 있는 제도보완을 청원,제안합니다",최근 1~ 2주 사이 우동기 대구시교육감에 대한 생각을 적지 않을 수 없어 몇 자 ...
...,...,...,...,...,...,...,...
7842,청원종료,195,2,2017-09-27,2017-10-27,대입 수능 성적 정보의 구체적이고 투명한 공개를 요청 드립니다.,대입 수험생을 둔 가정에서 많은 분들이 아쉬워하는 교육과정평가원의 수능 및 모의평가...
7843,청원종료,222,4,2018-04-18,2018-05-18,국회의원전수조사,이번국회의원피기감기관지원과관련전수조사하여잘못된관행을바로잡았으면합니다이렇게해야정치개혁...
7844,청원종료,211,1,2018-02-22,2018-03-24,안전하지 않는 원전 운영허가 취소를 해 주세요,울진 한울원전 4호기는 상업운전 초기부터 심각한 전열관 손상이 있었습니다. 그리고...
7845,청원종료,3343,2,2018-01-04,2018-02-03,서남대학교 특별편입 반대 단국대학교 천안캠퍼스.,여러분 혹시 지금 서남대학교 폐교에대한 기사를 보셨나요?\n\r\n죽전캠분들에겐 해...


In [6]:
train_data.shape

(7847, 7)

In [7]:
# 라벨링된 카테고리명 매핑 ex) {0: '경제민주화', 1: '교통/건축/국토', 2: '기타',3: '농산어촌', 4: '문화/예술/체육/언론'}
# mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
# mapping

In [8]:
# Train / Test set 분리
# from sklearn.model_selection import train_test_split
# train, test = train_test_split(new_data, test_size=0.2, random_state=42)
# train, test = train_test_split(new_data, test_size=0.2, random_state=42)
# print("train shape is:", len(train))
# print("test shape is:", len(test))

In [9]:
# train.to_csv('/content/drive/MyDrive/bigdata_deeplearning/data/train_data.csv', header = True, index = False)
# test.to_csv('/content/drive/MyDrive/bigdata_deeplearning/data/test_data.csv', header = True, index = False)

In [10]:
# 기본 Bert tokenizer 사용
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [11]:
class BERTDataset(Dataset):
    def __init__(self, dataset, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 
        self.sentences = [transform(i) for i in dataset['내용']]
        self.labels = [np.int32(i) for i in dataset['카테고리']]
    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
    def __len__(self):
        return (len(self.labels))
        

In [13]:
# Setting parameters
np.random.seed(100)
max_len = 30 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 124
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5


In [14]:
bert_train = BERTDataset(train_data, tok, max_len, True, False)
bert_test = BERTDataset(test_data, tok, max_len, True, False)

In [15]:
print(bert_train)
print(bert_test)

In [16]:
# pytorch용 DataLoader 사용
train_dataloader = torch.utils.data.DataLoader(bert_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(bert_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [17]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 28,
                 num_classes = 5, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()
    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [18]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [19]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [20]:
# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [21]:
# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()
    return train_acc
# /max_indices.size()[0]

In [ ]:
# # 모델 학습 시작
# for e in range(num_epochs):
#     train_acc = 0.0
#     # test_acc = 0.0
#     model.train()
#     for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
#         optimizer.zero_grad()
#         token_ids = token_ids.long().to(device)
#         segment_ids = segment_ids.long().to(device)
#         valid_length= valid_length
#         label = label.long().to(device)
#         out = model(token_ids, valid_length, segment_ids)
#         loss = loss_fn(out, label)
#         loss.backward
#         torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
#         optimizer.step()
#         scheduler.step()  # Update learning rate schedule
#         train_acc += calc_accuracy(out, label)
#         if batch_id % log_interval == 0:
#             print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
#     print("epoch {} train acc {}".format(e+1, train_acc / (7847)))

In [ ]:
# test

In [ ]:
# unseen_test = pd.DataFrame([[test_sentence, test_label]], columns = [['제목', '카테고리']])
# unseen_values = unseen_test.values
# test_set = BERTDataset(unseen_values, 0, 1, tok, max_len, True, False)
# test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=5)

In [22]:
test_input = torch.utils.data.DataLoader(test_data, batch_size=1, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# BERTDataset(unseen_values, tok, max_len, True, False)

In [ ]:
# tqdm_notebook(test_input)

In [ ]:
# # 모델 학습 시작
# test_acc = 0.0
# for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
#   optimizer.zero_grad()
#   token_ids = token_ids.long().to(device)
#   segment_ids = segment_ids.long().to(device)
#   valid_length= valid_length
#   label = label.long().to(device)
#   out = model(token_ids, valid_length, segment_ids)
#   loss = loss_fn(out, label)
#   loss.backward
#   torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
#   optimizer.step()
#   scheduler.step()  # Update learning rate schedule
#   test_acc += calc_accuracy(out, label)
#   if batch_id % log_interval == 0:
#     print("batch id {} loss {} test acc {}".format(batch_id+1, loss.data.cpu().numpy(), test_acc / (batch_id+1)))

In [23]:
# 모델 학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id == batch_size :
          print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id + 1)))    
    
# 

  0%|          | 0/64 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
  0%|          | 0/64 [00:00<?, ?it/s]

epoch 1 train acc 26.8125


  0%|          | 0/64 [00:00<?, ?it/s]

epoch 2 train acc 26.8125


  0%|          | 0/64 [00:00<?, ?it/s]

epoch 3 train acc 26.140625


  0%|          | 0/64 [00:00<?, ?it/s]

epoch 4 train acc 27.0625


100%|██████████| 64/64 [00:15<00:00,  4.15it/s]

epoch 5 train acc 26.8125


In [24]:
model.eval() # 평가 모드로 변경
test_acc = 0.0

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  label = label.long().to(device)
  out = model(token_ids, valid_length, segment_ids)
  test_acc += calc_accuracy(out, label)
  print("test acc {}".format(test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


test acc 33.0
test acc 37.5
test acc 36.0
test acc 32.75
test acc 31.2
test acc 31.333333333333332
test acc 31.142857142857142
test acc 30.75
test acc 30.77777777777778
test acc 30.6
test acc 29.90909090909091
test acc 29.75
test acc 31.076923076923077
test acc 31.285714285714285
test acc 31.133333333333333
test acc 30.5625

